<a href="https://colab.research.google.com/github/jk-math/Attendance_Tracker/blob/main/Attendance_Tracker_GH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import required libraries

In [ ]:
import os
import pandas as pd
import datetime
import numpy as np
import re

Provide a path to an Excel (.xlsx) or .csv file that will be used to create a class roster pandas dataframe. The files I use have columns titled ID and Name. The ID column is a unique student ID and the Name column is a string that needs to be split into first name, last name, and middle initial. The Name provided in Trinity roster is of the form Last,First MiddleInitial (if the student has middle initial).

In [ ]:
# Will be applied to the Name column in the roster
def split_name_roster(name_string):
  last, f_m = name_string.strip().split(',')
  try:
    first, middle = f_m.split(' ')
    return first, last, middle 
    # if name is like Smith,Bob Carl Joe R., this will fail, so the function will return first = Bob Carl Joe R.
    # no middle name will be returned in this case
  except:
    return f_m, last, ' '

In [ ]:
# provide path to a csv or xlsx file
# function will read file to a pandas dataframe
def read_file(path):
  is_csv = path.endswith('.csv')
  is_xlsx = path.endswith('.xlsx')

  if (not is_csv) and (not is_xlsx):
    print('File type {} not supported - upload an Excel workbook or CSV file'.format(path))
    return None
  elif is_csv:
    try:
      roster = pd.read_csv(path)
    except:
      print('Error: CSV file {} provided, but could not be read by Pandas'.format(path))
    return roster
  else:
    try:
      roster = pd.read_excel(path)
    except:
      print('Error: Excel file {} provided, but could not be read by Pandas'.format(path))
    return roster
  

In [ ]:
# Create csv file to track attendance from the roster provided by Trinity
# default is to save the attendance tracker to a csv file in my Drive folders
def create_attendance_roster(roster_path, save_path = '/content/drive/MyDrive/attendance/attendance_roster.csv', 
                             return_df = False, save_to_csv =True, use_time_buffer = False):

  extract_name_roster = np.vectorize(split_name_roster) # can apply function to entire column of a dataframe

  roster = read_file(roster_path)

  roster_cleaned = roster[['ID', 'Name']] # just grab Student ID and Name columns
  roster_cleaned = roster_cleaned.dropna() # Roster provided by Trinity has some completely blank rows
  roster_cleaned['ID'] = roster_cleaned['ID'].astype(int)

  F, L, M = extract_name_roster(roster_cleaned['Name'])
  roster_cleaned['First'] = F
  roster_cleaned['Last'] = L
  roster_cleaned['Middle'] = M
  roster_cleaned = roster_cleaned.drop('Name', axis = 1)
  roster_cleaned = roster_cleaned.set_index('ID')

  n_students = len(roster_cleaned.index)

  # create columns to track attendance
  roster_cleaned['On Time'] = np.zeros(n_students).astype(int)
  roster_cleaned['Late'] = np.zeros(n_students).astype(int)
  roster_cleaned['Absent'] = np.zeros(n_students).astype(int)
  if use_time_buffer:
    roster_cleaned['Even more Late'] = np.zeros(n_students).astype(int)

  if save_to_csv:
    try:
      roster_cleaned.to_csv(save_path)
    except:
      print('Error - make sure a valid file path is provided to save the roster to a csv file')

  if return_df:
    return roster_cleaned

In [ ]:
roster_path = '/content/drive/MyDrive/attendance/math207_roster.xlsx'

save_path = '/content/drive/MyDrive/attendance/math207_attendance_tracker.csv'

roster_df = create_attendance_roster(roster_path, save_path = save_path, 
                                     save_to_csv = True, return_df = True,
                                     use_time_buffer = True)

In [ ]:
roster_df

The attendance report from Teams does not have the student ID, so the function below will be used to create a dictionary where the keys are the student's full names in a single string and the values are the IDs.

In [ ]:
# create a dictionary where the keys are the students full names put into a single string
# the values are the student IDs
def name_id_dict(roster_path):
  rost = read_file(roster_path)
  return dict(zip(rost['First']+' '+ rost['Middle']+ ' '+ rost['Last'], rost['ID']))

The Microsoft Teams Attendance Report contains columns named Full Name, User Action (for example Joined or Left), and Timestamp (corresponding to the User Action). The earliest Timestamp indicates when the student joined the class. The student's name will not appear in the file if they did not attend class.

If the student logs in using their Trinity account, the Full Name column is of the form Last, First M. (ClassYear). There is no student ID column to uniquely identify students. If the student logs in via a different account, the Full Name entry will not be of this form. It will instead look like \\\\NameProvided (Guest)\"""" - this is for any person that attends the meeting without being in the scheduled meeting roster on Teams.



Make sure to save the Attendance report that you download from Teams as a .xlsx or .csv file. I have found that when the file appears in my Downloads folder, it is saved as a .txt file.

In [ ]:
def split_name_att(name_string, file_path = None):
  name_string.strip()
  name_string = name_string.strip('\\')
  name_string = name_string.strip('\"')
  if re.search('\(Guest\)', name_string): # indicates Guest in meeting
    names, other = re.split('\(Guest\)', name_string)
    names.strip() # names will be a string that is whatever name is provided by the Guest
    if file_path != None:
      print('Guest {} found in meeting {}'.format(names, file_path)) # this will identify the files where Guests appear
    return names, 'GUEST', 'GUEST' # need to decide how to handle Guests
  else:
    last, f_m_yr = name_string.strip().split(',') 
    f_m = f_m_yr[:-6].strip() # get rid of class year
    try:
      first, middle = f_m.split(' ')
      return first, last, middle 
      # if name is like Smith,Bob Carl Joe R., this will fail, so the function will return first = Bob Carl Joe R.
      # no middle name will be returned in this case
    except:
      return f_m, last, ' '

Change Timestamp info to a datetime object so we can do easy comparisons. The Timestamp is a string of the form 4/5/2021, 9:05:42 AM.

In [ ]:
def extract_datetime_info(timestamp_string):
  date, time = timestamp_string.split(',')
  time = time.lstrip()
  month, day, year = date.split('/')
  month, day, year = int(month), int(day), int(year)
  hour, min, sec_ = time.split(':')
  sec, a_or_p = sec_.split(' ')
  hour, min, sec = int(hour), int(min), int(sec)
  if hour == 12: # the hour parameter of datetime must be in range(24)
    hour = 0
  if a_or_p == 'AM':
    return datetime.datetime(year, month, day, hour = hour, minute = min, second = sec)
  else:
    return datetime.datetime(year, month, day, hour = hour+12, minute = min, second = sec)

In [ ]:
def update_attendance_tracker(att_path, roster_path,
                               instructors = ['Kreinbihl, James R.'], start_time = (8, 55),  # start time is hour, minute
                              time_buffer = (0,0), use_time_buffer = False): # time_buffer is minute, seconds
  att_report = read_file(att_path) # path to Teams attendance report file
  roster = read_file(roster_path) # file to an attendance tracker file that will be updated
  roster = roster.set_index('ID') # the attendance tracker we created has the ID column set as index
  # but when we save it to a csv file, the ID index just becomes a a column
  for instructor in instructors:
    att_report = att_report[att_report['Full Name'] != instructor] #remove instructors from report
  report = att_report.groupby('Full Name')[['Timestamp']].min().reset_index()
  # the min timestamp indicates time of arrival
  # note that as the instructor, the earliest time that will show as a timestamp is the time that I arrive
  # So, IF YOU ARRIVE LATE TO YOUR OWN CLASS, this will not accurately record attendance

  n_students = len(roster.index)

  extract_name_att = np.vectorize(split_name_att)

  F, L, M = extract_name_att(report['Full Name'], file_path = att_path)

  report['First'] = F
  report['Last'] = L
  report['Middle'] = M
  report = report.drop('Full Name', axis = 1)
  report = report.reset_index(drop=True)

  roster_names = name_id_dict(roster_path)

  att_names = report['First'] + ' ' + report['Middle'] + ' ' + report['Last']

  report['NameString'] = att_names # attendance report does not have IDs - so we will use NameString to uniquely identify students

  absent = set(roster_names.keys()) - set(att_names) # students in roster that don't show up in attendance report
  guests = set(att_names) - set(roster_names.keys()) # names in attendance report that are not in roster, ie Guests
  attended = set(att_names) - guests # students who attended, excluding Guests
  n_att = len(attended)

  for n in absent:
    roster.loc[int(roster_names[n]),['Absent']] += 1 

  extract_dt = np.vectorize(extract_datetime_info)

  report['Datetime'] = extract_dt(report['Timestamp'])

  report.drop('Timestamp',inplace = True, axis = 1)

  dt = report['Datetime'][0] # don't need to sort - just picking out day, month, year

  day, month, year = dt.day, dt.month, dt.year

  st = datetime.datetime(year, month, day, start_time[0], start_time[1], 0)

  report['Late'] = (report['Datetime'] > st).astype(int)
  # consider small time buffer for those students that arrive only seconds after the start time

  report['On Time'] = (report['Datetime'] <= st).astype(int)

  att_ids = []
  for n in attended:
    att_ids.append(roster_names[n])

  report.set_index('NameString', inplace=True)
  roster.sort_values(by = 'Last', axis = 0, inplace=True)
  report.sort_values(by = 'Last', axis = 0, inplace=True)

  roster.loc[att_ids,['Late']] += np.array(report.loc[attended, ['Late']])
  roster.loc[att_ids,['On Time']] += np.array(report.loc[attended, ['On Time']])

  if use_time_buffer:
    bst = datetime.datetime(year, month, day, start_time[0], start_time[1]+time_buffer[0], time_buffer[1])
    report['Even more Late'] = (report['Datetime'] > bst).astype(int)
    roster.loc[att_ids,['Even more Late']] += np.array(report.loc[attended, ['Even more Late']])


  roster.to_csv(roster_path) # this will override the original attendance tracking file passed to the function

  return roster

In [ ]:
base_dir = '/content/drive/MyDrive/attendance/Math207_attendance_reports' # Drive directory containing all Teams attendance reports
att_reps = os.listdir('/content/drive/MyDrive/attendance/Math207_attendance_reports/') # list all file names for attendance reports
for r in att_reps:
  update_attendance_tracker(att_path = os.path.join(base_dir,r),
                             roster_path = '/content/drive/MyDrive/attendance/math207_attendance_tracker.csv',
                            use_time_buffer = True,
                            time_buffer = (1,0))


In [ ]:
pd.read_csv('/content/drive/MyDrive/attendance/math207_attendance_tracker.csv')